# VibraCore report

______________________________________________________________________

**Authors:** \
Robin Wimmers\
Jorrit de Vries

Notebook with stepwise explanation of reading and processing soil and building data and writing the results to a standard template. For more information about VibraCore, take a look at the documentation!

The notebook is organized in the following sections:

1. [Project Information](#Project-Information)
1. [Create Dataframe with Buildings](#Create-Dataframe-with-Buildings)
1. [Results](#Results)
1. [Sound](#Sound)
1. [Writing and Downloading Report](#Writing-and-Downloading-Report)

In [ ]:
import os
from typing import Dict
import io
import logging

import pygef
from ipywidgets import FileUpload
from IPython.display import display
import fiona
from nuclei.client import NucleiClient
from pyvibracore import api
from pyvibracore.input.vibration_properties import (
    create_cur166_payload,
    create_prepal_payload,
    get_buildings_geodataframe,
    get_normative_building,
    create_single_payload,
    create_vibration_report_payload,
)
from pyvibracore.results.vibration_result import (
    plot_reduction,
    VibrationResults,
    map_payload,
    plot_reduction,
)
from pyvibracore.results.nuisance import map_nuisance, df_nuisance, CFC_FACTOR_FLOORS
from pyvibracore.results.sound_result import (
    get_normative_building as get_normative_building_sound,
)
from pyvibracore.results.sound_result import map_sound
from shapely.geometry import LineString, Polygon, Point
import geopandas as gpd
import pandas as pd
import contextily as ctx

from tqdm import tqdm

pd.set_option("display.max_columns", None)
logging.getLogger().setLevel(logging.INFO)

### Start a Nuclei client session

In the next cell we will create a nuclei-client with a session that takes care of the
authentication and communication with the Nuclei server.

You will need to provide your user token, which can be obtained by login in to the [nuclei website](https://nuclei.cemsbv.io/) with your personal credentials and going to the "API Access Tokens" section.

In [ ]:
# os.environ["NUCLEI_TOKEN"] = "<YOUR TOKEN>"

client = NucleiClient()

# Project Information

Below standard information of the project is gathered:

<div style="background: #f2ed4c;
            width: 100%;
            color: black;
            text-align: center;">
<b>USER INPUT REQUIRED BELOW:<b>
</div>

In [ ]:
# General input data
project_id = 21305  # also used to select cpt
project_remark = "Voorbeeld Notebook"  # Optional additional information
author = "N. Uclei"
project_name = "Vibration prediction"

# ** methode:
# Vibration prediction methode
# Accepted values: ["prepal", "cur166"]
methode = "prepal"

In the cell below you add the coordinates used to download the buildings from the BAG. Please add the coordinates so that you have a nice square within which all buildings are downloaded. No need to make the perfect sized square, later on you can delete the buildings you don't want to analyze.

In [ ]:
# Points in RD_new
(west, south, east, north) = (120388.1, 486204.7, 120653.0, 486357.3)

# define project location can be file path
project_location = "./constructionSite.geojson"

In [ ]:
"""
Characteristics SBR-A
"""

# ** category:
# Building category based on the SBR A table 10.1.
# Accepted values: ["one", "two"]
category = "two"

# ** frequency:
# The dominate frequency [Hz]
frequency = 30.0  # Hz

# ** vibration_type:
# Based on the SBR A table 10.4.
# Accepted values: ['short-term', 'repeated-short-term', 'continuous']
vibration_type = "continuous"

# ** vibration_sensitive
# Has the building structure a vibration sensitive
# foundation. Based on the SBR A chapter 10.2.5
# Accepted values: [True, False]
vibration_sensitive = False

# ** structural_condition
# Based on the SBR A table 10.2.
# Accepted values: ['sensitive', 'normal']
structural_condition = "normal"

# **frequency_vibration_sensitive
# The dominate frequency for vibration sensitive building [Hz].
# Prepal default: 20Hz; CUR166 default: 40Hz
frequency_vibration_sensitive = 40.0  # Hz

# ** thickness
# Layer thickness settlement-sensitive layer [m]
# range: 1 - 8
thickness = 8.0

# ** referenceLocation
# Based on CUR 166-1997 table 5.16 and 5.17
# Accepted values: ["Amsterdam", "Maasvlakte", "Rotterdam", "Groningen", "Den Haag", "Tiel", "Eindhoven"]
reference_location = "Amsterdam"

# ** monumental
# Has the building structure a monumental status. Based on the SBR A table 10.3.
# Accepted values: ["False", "True" ]
monumental = False

# ** measurement_type
# Type of measurement based on the SBR A table 9.2.
# Accepted values: ["indicative", "limited", "extensive" ]
measurement_type = "extensive"

In [ ]:
"""
Characteristics CUR166
"""

# ** material_floor
# Based on CUR 166 3rd edition table 5.20
# Accepted values: ["concrete", "wood" ]
material_floor = "concrete"

# ** installation_type
# Based on the SBR A table 10.4.
# Accepted values: ["vibrate", "driving" ]
installation_type = "vibrate"

# ** building_part
# Based on CUR 166 3rd edition table 5.20 or 5.21
# Accepted values: ["floor", "wall" ]
building_part = "floor"

# ** building_part
# Based on CUR 166 3rd edition table 5.19
# Accepted values: ["shallow foundation", "concrete piles", "timber piles", "steel piles" ]
foundation_element = "concrete piles"

# ** safety_factor
# CUR166 overschreidingskans
safety_factor = 0.05  # safety_factor = CUR166 overschreidingskans

# ** vibration_direction
# Based on CUR 166 3rd edition table 5.22
# Accepted values: ["vertical", "horizontal"]
vibration_direction = "vertical"

# ** force
# impact force [kN]
force = 1350

# ** force_reduction
# factor 5% wanneer trilblok wordt gebruikt,
# factor 10% - 20% wanneer heiblok wordt gebruikt,
# 0% wanneer slagkracht berekend obv conusweerstand.
force_reduction = 0

# ** methode_safety_factor
# Parameter that indicated how the safety factor is calculated.
# Accepted values: ["CUR", "exact" ]
methode_safety_factor = "exact"

In [ ]:
"""
Characteristics PrePal
"""
# ** heireductie_factor
# 1, inwendig heien = 0.3 - 0.5 [-]
heireductie_factor = 1

# ** cone_resistance
# Characteristics cone resistance [MPa]
cone_resistance = 36

# ** cone_resistance_reduction
# Reduction of the cone resistance [%]
cone_resistance_reduction = 0.0

# ** unit_weight
# soil unit weight [kN/m3]
unit_weight = 20

# ** elastic_modulus_factor
# Elastic modulus factor of the soil [-].
elastic_modulus_factor = 15

# ** poisson_ratio
# Poisson’s ratio of the soil [-]
poisson_ratio = 0.2

# ** pile_shape
# Shape of the pile.
# Accepted values: ["square", "round"]
pile_shape = "square"

# ** pile_size
# Size of the pile [m]
# Single side length of the piles, diameter when pile pile is round
pile_size = 220e-3

# ** hysteretic_damping_barkan
# hysteretic damping barkan [m^-1]
hysteretic_damping_barkan = -0.05

In [ ]:
"""
Characteristics Sound
"""
# ** power
# source power [dB]
power = 100

# ** k2
# Correction term [dB]
k2 = 5

# ** period
# Operating period of the building code [hours]
period = 5

<div style="background: #f2ed4c;
            width: 100%;
            color: black;
            text-align: center;">
<b>END USER INPUT<b>
</div>

In [ ]:
# Location of the piles

if not os.path.isfile(project_location):
    raise FileExistsError(
        f"the project location file {project_location} does not exists."
        "Please make sure that the file is in the correct folder."
    )

location = (
    gpd.read_file(project_location).explode(index_parts=False)["geometry"].values
)[0]
location

# Create Dataframe with Buildings
This notebook creates a dataframe with the vibration source, surrounding buildings and default parameters for these buildings. The cell below creates and downloads this dataframe as Geojson from the BAG. To change any of the parameters, delete or add buildings and check if it is correct, upload the Geojson to QGIS. You can alter, by hand, all the parameters. After finishing you can upload the geojsn with the uplaod button below.

In [ ]:
upload = FileUpload(accept=".geojson", multiple=False)
buildings = get_buildings_geodataframe(
    west,
    south,
    east,
    north,
    category,
    monumental,
    structural_condition,
    vibration_sensitive,
    thickness,
    foundation_element,
    material_floor,
)

# save building data to file
buildings.to_file("dataframe.geojson", driver="GeoJSON")

<div style="background: #FFA500;
            width: 100%;
            color: black;
            text-align: center;">
<b>After uploading you own file, please make sure that you (re)run all the code blocks below. <br/>
    You can use the `run>Run Selected Cell and All Below` from the menu bar to run all code blocks<b>
</div>

In [ ]:
display(upload)

In [ ]:
if len(upload.value) == 1:
    logging.info(f"Use uploaded geojson file. File name: {upload.value[0].name}")
    content = upload.value[0].content.tobytes()
    with fiona.BytesCollection(content) as f:
        crs = f.crs
        buildings = gpd.GeoDataFrame.from_features(f, crs=crs).explode(
            index_parts=False
        )

fig = map_payload(buildings, location)

# add basemap
ctx.add_basemap(fig.axes[0], crs="EPSG:28992", source=ctx.providers.Esri.WorldTopoMap)

buildings[
    [
        "name",
        "monumental",
        "category",
        "structuralCondition",
        "vibrationSensitive",
        # "foundationElement",
        # "material",
        # "thickness",
        # "buildingDepth",
        # "buildingDepthVibrationSensitive",
        "bouwjaar",
        "gebruiksdoel",
        "status",
    ]
].head(10)

# Create Classes
Below standard information of the project is gathered:

In [ ]:
if methode.lower() == "prepal":
    multi_vibration_payload = create_prepal_payload(
        buildings,
        location,
        pile_shape,
        pile_size,
        cone_resistance,
        cone_resistance_reduction,
        unit_weight,
        elastic_modulus_factor,
        poisson_ratio,
        frequency,
        vibration_type,
        frequency_vibration_sensitive,
        measurement_type,
        hysteretic_damping_barkan,
    )

    response = api.get_prepal_calculation(client, payload=multi_vibration_payload)
elif methode.lower() == "cur166":
    multi_vibration_payload = create_cur166_payload(
        buildings,
        location,
        force,
        force_reduction,
        installation_type,
        building_part,
        safety_factor,
        vibration_direction,
        frequency,
        vibration_type,
        frequency_vibration_sensitive,
        reference_location,
        measurement_type,
        methode_safety_factor,
    )

    response = api.get_cur166_calculation(client, payload=multi_vibration_payload)
else:
    raise NotImplementedError(
        f"Methode {methode} not implemented. Accepted values: [prepal, cur166]"
    )

results = VibrationResults.from_api_response(response)

# Results
To get an overview of the result of the vibration calculation run the code-block below.

In [ ]:
fig = results.map(location)

# add basemap
ctx.add_basemap(fig.axes[0], crs="EPSG:28992", source=ctx.providers.Esri.WorldTopoMap)

In [ ]:
name = get_normative_building(buildings, location, category="two")
if name:
    single_payload = create_single_payload(multi_vibration_payload, name=name)
    result = client.call_endpoint(
        "VibraCore", f"/{methode.lower()}/validation/single", schema=single_payload
    )
    plot_reduction(result, sensitive=True);

In [ ]:
name = get_normative_building(buildings, location, category="one")
if name:
    single_payload = create_single_payload(multi_vibration_payload, name=name)
    result = client.call_endpoint(
        "VibraCore", f"/{methode.lower()}/validation/single", schema=single_payload
    )
    plot_reduction(result, sensitive=True);

# SBR-B

In [ ]:
name = get_normative_building_sound(buildings, location)

if name:
    single_payload = create_single_payload(multi_vibration_payload, name=name)
    result = client.call_endpoint(
        "VibraCore", f"/{methode.lower()}/validation/single", schema=single_payload
    )
    cfc = CFC_FACTOR_FLOORS[installation_type][
        buildings.loc[buildings["name"] == name]["material"].item()
    ]["Cfc"]
    fig = map_nuisance(
        buildings,
        source_location=location,
        building_name=name,
        response_dict=result,
        cfc=cfc,
        u_eff=0.54,
        period=period,
    )

    # add basemap
    ctx.add_basemap(
        fig.axes[0], crs="EPSG:28992", source=ctx.providers.Esri.WorldTopoMap
    )

    print(df_nuisance(response_dict=result, cfc=cfc, u_eff=0.54, period=period))

# Sound
To get an overview of the result of the sound calculation run the code-block below.

In [ ]:
name = get_normative_building_sound(buildings, location)
if name:
    fig = map_sound(
        buildings, location, building_name=name, power=power, k2=k2, period=period
    )

    # add basemap
    ctx.add_basemap(
        fig.axes[0], crs="EPSG:28992", source=ctx.providers.Esri.WorldTopoMap
    )

# Writing and Downloading Report

The following code wil call the report API.

In [ ]:
# Create report
report_payload = create_vibration_report_payload(
    multi_vibration_payload,
    project_name=project_name,
    project_id=str(project_id),
    author=author,
)

if methode.lower() == "prepal":
    report = api.get_prepal_report(client=client, payload=report_payload)
elif methode.lower() == "cur166":
    report = api.get_cur166_report(client=client, payload=report_payload)
else:
    raise NotImplementedError(
        f"Methode {methode} not implemented. Accepted values: [prepal, cur166]"
    )

with open(f"{project_name} report.pdf", "wb") as f:
    f.write(report)